In [1]:
import MetaTrader5 as mt5
from datetime import datetime
import pytz
import pandas as pd 
import matplotlib.pyplot as plt 

set up connections with MT5. It requires MT5 to be opened, Otherwise it will auto open MT5.  
+ the MetaTrader5 should be installed in your laptop, the path is the directory to the terminal
+ login and password are the credentials. (this account is a virtual account with 10k USD and 1:25 leverage)   

Query data from MT5. it can be query through different timestep.   
+ Data need to be in datetime format, need to convert the end time to a UTC standard time.   
+ Data can be extracted in various forms.   
+ ** mt5.TIMEFRAME_H4 ** : is extracting each data point in 4-hour time length.   
+ more can be found: [MT5 documentation](https://www.mql5.com/zh/docs/integration/python_metatrader5/mt5copyratesfrom_py)
        
Remember to shutdown connections after query

In [2]:
#build connection
mt5.initialize(
    path = "C:\\Users\\silin\\AppData\\Roaming\\MetaTrader 5\\terminal64.exe",
    login = 35146240,
    password = "xt7lixru"
    )

# query data
timezone = pytz.timezone("Etc/UTC")
utc_from = datetime(2020, 9, 23, tzinfo=timezone)

rates = mt5.copy_rates_from('EURUSD', mt5.TIMEFRAME_H4, datetime.today(), 2200)

#off connection
mt5.shutdown()

True

This part is converting data into a Pandas format

In [3]:
# cleanup data
rates_frame = pd.DataFrame(rates)
rates_frame['time']=pd.to_datetime(rates_frame['time'], unit='s')
#rates_frame.to_csv("EURUSD_200923_190425.csv")

Codes to draw candle stick chart (Working In Progress)

In [4]:
plt_df = rates_frame[['time', 'open', 'high', 'low', 'close']]

In [ ]:
import plotly.graph_objects as go


fig = go.Figure(data=[go.Candlestick(x=plt_df['time'],
                open=plt_df['open'],
                high=plt_df['high'],
                low=plt_df['low'],
                close=plt_df['close'])])

fig.show()

This section is showing how to excute orders. [Ref](https://www.mql5.com/zh/docs/integration/python_metatrader5/mt5ordersend_py)

In [ ]:
# open an order

mt5.initialize()

symbol = "USDJPY"               # trade pairs 
symbol_info = mt5.symbol_info(symbol)

if not symbol_info.visible:
    print(symbol, "is not visible, trying to switch on")
    if not mt5.symbol_select(symbol,True):
        print("symbol_select({}}) failed, exit",symbol)
        mt5.shutdown()
        quit()

lot = 0.1          # 0.1 lot (deposit 400 USD on 1:25 lvg)
point = mt5.symbol_info(symbol).point
price = mt5.symbol_info_tick(symbol).ask   # this is a sample buy order
deviation = 20                              # execute price acceptable range 
request = {
    "action": mt5.TRADE_ACTION_DEAL,
    "symbol": symbol,
    "volume": lot,
    "type": mt5.ORDER_TYPE_BUY,
    "price": price,
    "sl": price - 100 * point,
    "tp": price + 100 * point,
    "deviation": deviation,
    "magic": 234000,
    "comment": "python script open",
    "type_time": mt5.ORDER_TIME_GTC,
    "type_filling": mt5.ORDER_FILLING_RETURN,
}

result = mt5.order_send(request)

print("1. order_send(): by {} {} lots at {} with deviation={} points".format(symbol,lot,price,deviation));
if result.retcode != mt5.TRADE_RETCODE_DONE:
    print("2. order_send failed, retcode={}".format(result.retcode))

In [41]:
# close above order

lot = 0.1          
point = mt5.symbol_info(symbol).point
price = mt5.symbol_info_tick(symbol).bid   # change to bid
deviation = 20                              

request = {
    "action": mt5.TRADE_ACTION_DEAL,
    "symbol": symbol,
    "volume": lot,
    "type": mt5.ORDER_TYPE_SELL,           # change to SELL
    "price": price,
    #"sl": price - 100 * point,
    #"tp": price + 100 * point,
    "deviation": deviation,
    "magic": 234000,
    "comment": "python script close",     # change to close
    "type_time": mt5.ORDER_TIME_GTC,
    "type_filling": mt5.ORDER_FILLING_RETURN,
}

result = mt5.order_send(request)

print("1. order_send(): by {} {} lots at {} with deviation={} points".format(symbol,lot,price,deviation));
if result.retcode != mt5.TRADE_RETCODE_DONE:
    print("2. order_send failed, retcode={}".format(result.retcode))

1. order_send(): by USDJPY 0.1 lots at 105.389 with deviation=20 points


A sample code from Stackoverflow : open and close an order (haven't validate)

In [13]:
def get_info(symbol):
    '''https://www.mql5.com/en/docs/integration/python_metatrader5/mt5symbolinfo_py
    '''
    # get symbol properties
    info=mt5.symbol_info(symbol)
    return info

def open_trade(action, symbol, lot, sl_points, tp_points, deviation):
    '''https://www.mql5.com/en/docs/integration/python_metatrader5/mt5ordersend_py
    '''
    # prepare the buy request structure
    symbol_info = mt5.symbol_info(symbol)

    if action == 'buy':
        trade_type = mt5.ORDER_TYPE_BUY
        price = mt5.symbol_info_tick(symbol).ask
    elif action =='sell':
        trade_type = mt5.ORDER_TYPE_SELL
        price = mt5.symbol_info_tick(symbol).bid
    point = mt5.symbol_info(symbol).point

    buy_request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": trade_type,
        "price": price,
        "sl": price - sl_points * point,
        "tp": price + tp_points * point,
        "deviation": deviation,
        "magic": ea_magic_number,
        "comment": "sent by python",
        "type_time": mt5.ORDER_TIME_GTC, # good till cancelled
        "type_filling": mt5.ORDER_FILLING_RETURN,
    }
    # send a trading request
    result = mt5.order_send(buy_request)        
    return result, buy_request 


def close_trade(action, buy_request, result, deviation):
    '''https://www.mql5.com/en/docs/integration/python_metatrader5/mt5ordersend_py
    '''
    # create a close request
    symbol = buy_request['symbol']
    if action == 'buy':
        trade_type = mt5.ORDER_TYPE_BUY
        price = mt5.symbol_info_tick(symbol).ask
    elif action =='sell':
        trade_type = mt5.ORDER_TYPE_SELL
        price = mt5.symbol_info_tick(symbol).bid
    position_id=result.order
    lot = buy_request['volume']

    close_request={
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_SELL,
        "position": position_id,
        "price": price,
        "deviation": deviation,
        "magic": ea_magic_number,
        "comment": "python script close",
        "type_time": mt5.ORDER_TIME_GTC, # good till cancelled
        "type_filling": mt5.ORDER_FILLING_RETURN,
    }
    # send a close request
    result=mt5.order_send(close_request)
